In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()

In [ ]:
from aiida.manage.configuration import get_profile

from aiida.common.datastructures import CalcInfo, CodeInfo
from aiida.common.folders import Folder

from aiida.engine import CalcJob, WorkChain
from aiida.engine import calcfunction, workfunction, submit, run

from aiida.plugins import DataFactory, WorkflowFactory

from aiida.parsers.parser import Parser
from aiida.orm import Code
from aiida.orm import load_code, load_node
from aiida.orm import Str, Dict, Float, Int
from aiida.orm import QueryBuilder

import aiida

from alamode_aiida.data_loader import load_anphon_kl, load_anphon_kl_spec
from ase.io.espresso import write_espresso_in
from ase.io.lammpsdata import read_lammps_data
from ase.build import make_supercell
from ase import Atom, Atoms
import ase

from itertools import combinations_with_replacement
import numpy as np
import os
import subprocess
import shutil
import re
from time import sleep
import spglib
import json

from os.path import expanduser

from tools import NodeBank
from tools import wait_for_node_finished
from tools import load_atoms
from tools.putstructure import PutStructure


In [ ]:
# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
g_graph = False

with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path

In [ ]:
# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)

In [ ]:
# 計算するroot directoryの定義
key = "root_wd"
g_root_wd = nodebank.load_or_dump(key, Str(os.path.join(os.getcwd(), CWD)))
os.makedirs(g_root_wd.value, exist_ok=True)
g_root_wd


In [ ]:
# 構造ファイルの取得
# lammpsから取得する場合の例
_path = os.path.abspath("lammps_input/Si222/Si222.lammps")
g_structure_org_filename = Str(_path)
g_file_format = Str("lammps-data")
nodebank.dump("structure_org_format", g_file_format)
g_file_style = Str("atomic")
nodebank.dump("structure_org_style", g_file_style)

g_raw_structure = load_atoms(
    g_structure_org_filename, g_file_format, g_file_style)
g_raw_structure.get_ase()

In [ ]:
# supercellをファイルの構造の何倍にするかの定義をする
g_diag = ArrayData()
g_diag.set_array('diag', np.array([1, 1, 1]))
g_diag = nodebank.load_or_dump("diag", g_diag)
g_diag

In [ ]:
g_file_format = Str("lammps-data")
g_primcell_filename = Str(f"primcell.lammps")
g_supercell_filename = Str(f"supercell.lammps")
inputs = {"cwd": g_root_wd, "diag": g_diag, "output_format": g_file_format,
          "primcell_filename": g_primcell_filename,
          "supercell_filename": g_supercell_filename,
          "structure": g_raw_structure}
g_primsuper_structures = run(PutStructure, **inputs)


In [ ]:
g_primsuper_structures  # outout

In [ ]:
# filepath
g_primstructure_filepath = g_primsuper_structures["primstructure_filepath"]
# filepath
g_superstructure_filepath = g_primsuper_structures["superstructure_filepath"]

In [ ]:
g_superstructure = g_primsuper_structures["superstructure"]  # StructureData
g_primstructure = g_primsuper_structures["primstructure"]  # StructureData

In [ ]:
# StructureData
g_superstructure_file = g_primsuper_structures["superstructure_file"]

In [ ]:
if False:
    pk = g_ciffilename.pk
    print(pk)
    !verdi node graph generate $pk

In [ ]:
nodebank.dump("primstructure_filepath", g_primstructure_filepath)
nodebank.dump("superstructure_filepath", g_superstructure_filepath)
nodebank.dump("superstructure_file", g_superstructure_file)
nodebank.dump("primstructure", g_primstructure)
nodebank.dump("superstructure", g_superstructure)

In [ ]:
# ファイルを置くsubdirectory名の作成
# ディレクトリ名 cwd=formula_supercell_harmonic
# ディレクトリがなければ作成する。
g_almprefix = "_".join([g_primstructure.get_formula(), "x".join(
    map(str, g_diag.get_array("diag"))), "harmonic"])
g_almprefix = Str(g_almprefix)
g_cwd = Str(os.path.join(g_root_wd.value, g_almprefix.value))
g_cwd = nodebank.load_or_dump("cwd", g_cwd)
os.makedirs(g_cwd.value, exist_ok=True)
g_cwd

In [ ]:
nodebank.dump("almprefix", g_almprefix)